In [1]:
from scripts import Analysis, Decision, Plots
import subprocess
## Bard as LLM
import os
import requests
from bardapi.constants import SESSION_HEADERS
from bardapi import Bard

token = "g.a000gAjl7UgbfiwQeipmuDl3L5hWIpVrgNYv4uImLhBVCFADJ0i1NXUnSOvbamut_3LxhIgW8wACgYKAUUSAQASFQHGX2MiE3fzDYKMmJPOoLpjrGSUIxoVAUF8yKqCrfbBLjh19mu2iiaqHJC_0076"

session = requests.Session()
session.headers = SESSION_HEADERS
session.cookies.set("__Secure-1PSID", token)
#session.cookies.set("__Secure-1PSIDTS", "<VALUE>")
session.cookies.set("__Secure-1PSIDCC", "ABTWhQHfyr09QRhjV-LI650p4kdTKVi5gbUsy8aTmnoV7XvABYtTssiMaJNwEmoExs8Zfg6m")


bard = Bard(token=token, session=session)

/home/dm/miniconda3/envs/ice_ae/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib
/home/dm/miniconda3/envs/ice_ae/lib/python3.11/site-packages/MDAnalysis/analysis/data/filenames.py:110: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename


__Secure-1PSID value should end with a single dot. Enter correct __Secure-1PSID value.


In [2]:
top = "./Data/top.gro"
traj = "./Data/run_0.xtc"

In [3]:
# For one trajectory..
ana = Analysis(traj, top = top)
# RMSD analysis..
rmsd = ana.RMSD()
# ROG analysis..
rog = ana.ROG()
# SASA analysis..
sasa = ana.SASA()
############### Decesion based on the above data..
decision = Decision()
message_str = ""
message_str += decision.number_of_transitions_from_rmsd(rmsd)
message_str += decision.decideROG(ana.time, rog)
message_str += decision.decideSASA(ana.time, sasa)

In [4]:
############# Bard to assemble and rewrite things in nice way
prompt = "Following is MD simulation trajectory analysis result. Make thoughtful report of the below statements. write some thing that is copiable, and please don't write any thing in first person mode:"

result = bard.get_answer(prompt + message_str)
bard_output = (result['content'])
splitted_out = bard_output.split("\n")
refined_text = ""
for line in splitted_out:
    if "**Note" not in line:
        refined_text += line + "\n"


In [7]:
######## Plotting part
out = "./Out/"
plots = Plots(out = out)
plots.plotROG(ana.time, rog)
plots.plotRMSD(ana.time, rmsd)
plots.plotSASA(ana.time, sasa)

In [8]:
pymol_script =f"""load {top}
load_traj {traj}
hide everything 
show cartoon
show cell
set cgo_line_width, 2
set_view (\
     1.000000000,    0.000000000,    0.000000000,\
     0.000000000,    1.000000000,    0.000000000,\
     0.000000000,    0.000000000,    1.000000000,\
     0.000000000,    0.000000000, -149.570785522,\
    24.722297668,   24.784605026,   25.375907898,\
   111.117927551,  188.023696899,   20.000000000 )
beautify
ray 
png {out}init
frame {int(len(ana.time))}
ray 
png {out}final
quit"""
with open("generate_init_final.pml", "w") as f:
    f.write(pymol_script)

popen = subprocess.Popen(['/home/dm/Soft/pymol/pymol', 'generate_init_final.pml', '-cq'], stdout=subprocess.PIPE)


In [9]:
from md2pdf.core import md2pdf

In [14]:
md2pdf(f"{out}test.pdf",md_content = refined_text)

In [37]:
class printColor:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [45]:
pc = printColor()
def welcome():                                                                                
    base_str =   printColor.BLUE + printColor.BOLD + f"""
{printColor.RED + printColor.BOLD}ooooooooooooo                              .oooooo.{printColor.END}                          
{printColor.GREEN + printColor.BOLD}8'   888   `8                             d8P'  `Y8b{printColor.END}                         
     {printColor.YELLOW + printColor.BOLD}888      oooo d8b  .oooo.           888            .ooooo.  ooo. .oo.{printColor.END}   
     888      `888""8P `P  )88b          888           d88' `88b `888P"Y88b  
     888       888      .oP"888  8888888 888     ooooo 888ooo888  888   888  
     888       888     d8(  888          `88.    .88'  888    .o  888   888  
    o888o     d888b    `Y888""8o          `Y8bood8P'   `Y8bod8P' o888o o888o 
                                                                                                                                        
                                                                             
"""+printColor.END
    base_str += f"\t\t... a tiny {printColor.BLUE + printColor.BOLD }AI{printColor.END} assitant who helps you analyze your {printColor.GREEN + printColor.BOLD }MD simulated trajectory.{printColor.END}"
    return base_str

In [46]:
message = welcome()
print(message)


ooooooooooooo                              .oooooo.                          
8'   888   `8                             d8P'  `Y8b                         
     888      oooo d8b  .oooo.           888            .ooooo.  ooo. .oo.   
     888      `888""8P `P  )88b          888           d88' `88b `888P"Y88b  
     888       888      .oP"888  8888888 888     ooooo 888ooo888  888   888  
     888       888     d8(  888          `88.    .88'  888    .o  888   888  
    o888o     d888b    `Y888""8o          `Y8bood8P'   `Y8bod8P' o888o o888o 
                                                                                                                                        
                                                                             
		... a tiny AI assitant who helps you analyze your MD simulated trajectory.
